In [1]:
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig
)
from peft import (
    LoraConfig, 
    get_peft_model, 
    prepare_model_for_kbit_training, 
    TaskType, 
    PeftModel
)
from datasets import Dataset
import torch
import json
import re

# -------------------------------
# 1️⃣ 경로 및 모델명 설정
# -------------------------------
file_path = "train_data/cleaned_dataset_half.json"
base_model = "beomi/KoAlpaca-Polyglot-5.8B"
lora_model_path = "train_data/beomi_llm"

# -------------------------------
# 2️⃣ 데이터 로드
# -------------------------------
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)

# -------------------------------
# 3️⃣ 토크나이저 설정 및 토크나이징 함수 정의
# -------------------------------
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    prompt = example["input"].strip()
    response = example["output"].strip()
    full_text = prompt + "\n" + response

    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize, remove_columns=["input", "output"])


/Users/won3956/Desktop/DSBA_project1/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 6353/6353 [00:01<00:00, 4030.72 examples/s]


In [4]:
# -------------------------------
# 4️⃣ LoRA 설정 및 모델 준비 (MPS 전용, bitsandbytes 없이)
# -------------------------------
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map={"": device}
)

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["attention.query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# LoRA 모델 적용
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model = PeftModel.from_pretrained(model, lora_model_path)
model.to(device)  # MPS에 모델 이동
model.eval()


Fetching 13 files:   0%|          | 0/13 [02:34<?, ?it/s]


KeyboardInterrupt: 